# Assignment 3: Hidden Markov Models

---

## Task 1) Isolated Word Recognition

In this assignment, we'll be revising word recognition, this time using Hidden Markov Models (HMM).
As with [assignment 1](https://github.com/seqlrn/assignments/tree/master/1-dynamic-programming), we'll be using the [free spoken digits](https://github.com/Jakobovski/free-spoken-digit-dataset) dataset.
We will be using the [`pandas`](https://pandas.pydata.org/docs/) library for data handling and [`hmmlearn`](https://hmmlearn.readthedocs.io/en/latest/index.html) library for HMMs which depends on `numpy`.
Install the `pandas` and `hmmlearn` packages in your working environment and get familiar with these modules.


### Data

Download Zohar Jackson's [free spoken digit](https://github.com/Jakobovski/free-spoken-digit-dataset) dataset.
There's no need to clone, feel free to use a revision, like [v1.0.10](https://github.com/Jakobovski/free-spoken-digit-dataset/archive/refs/tags/v1.0.10.tar.gz).
The file naming convention is `{digitLabel}_{speakerName}_{index}.wav`.

### Basic Setup

As you can learn from the [tutorial](https://hmmlearn.readthedocs.io/en/latest/tutorial.html#), `hmmlearn` provides us with the base implementation of Hidden Markov Models; we'll be using the `hmm.GaussianHMM`, which implements HMMs with a single Gaussian emission probability per state.
For a starter, build a basic isolated word recognizer that uses a separate model for each digit.

*In this Jupyter Notebook, we will provide the steps to solve this task and give hints via functions & comments. However, code modifications (e.g., function naming, arguments) and implementation of additional helper functions & classes are allowed. The code aims to help you get started.*

---

In [ ]:
# Dependencies
import os
import librosa
import numpy as np
import pandas as pd
from hmmlearn import hmm

### Prepare the Data

1.1 To facilitate the selection of samples for speakers and digits, consider how you can store the data within a `pandas.DataFrame`.

1.2 Compute the MFCC features for the complete data set (3000 recordings; use `n_mfcc=13`).

1.3 Apply per-speaker feature normalization (e.g., standardization).

In [ ]:
NUM_SAMPLES = 50 # recordings per speaker & digit
DIGITS = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
SPEAKERS = ["george", "jackson", "lucas", "nicolas", "theo", "yweweler"]

In [ ]:
### Notice: a good default value is 25ms for FFT window and 10ms for hop length
### Notice: be careful as librosa takes the number of samples as input!        

def compute_features(file):
    """Computes the features for a recording file."""
    ### YOUR CODE HERE
    
    raise NotImplementedError()
    
    ### END YOUR CODE


def load_dataframe(input_dir):
    """Loads the recordings into a pandas.DataFrame."""
    ### YOUR CODE HERE
    
    raise NotImplementedError()
    
    ### END YOUR CODE


def normalize_features(dataframe):
    """Applies per-speaker feature normalization."""
    ### YOUR CODE HERE
    
    raise NotImplementedError()
    
    ### END YOUR CODE

In [ ]:
INPUT_DIR = "<path to your recordings>"
dataframe = load_dataframe(input_dir=INPUT_DIR)
dataframe_w_norm = normalize_features(dataframe=dataframe)

### Notice: just for test purposes

# print("Num recordings: {}".format(len(dataframe)))
# for speaker in SPEAKERS:
#     print("### {}".format(speaker))
#     data_speaker = dataframe[dataframe["speaker"] == speaker]

#     print(data_speaker["digit"].value_counts())
#     print()

### Train and Evaluate

2.1 Implement a 6-fold [cross-validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)) loop for the 6 speakers to (later) figure out, which test speaker performs best/worst. That is, each speaker acts as test speaker while the others are used for training (with each possible combination).

2.2 Inside the cross-validation loop, train an individual HMM with linear topology for each digit. There are several points to consider:

*The [`fit`](https://github.com/hmmlearn/hmmlearn/blob/38b3cece4a6297e978a204099ae6a0a99555ec01/lib/hmmlearn/base.py#L439) expects features to be sequential in a single array with `X` as (n_train_samples, n_features). Furthermore, we need to pass the lengths of each recording into the function with`lengths` as (n_samples,):*

```python
### you can flatten the features of the train data as follows

# input: [(rec_samples_1, n_feats), ..., (rec_samples_N, n_feats)]
# output: (all_rec_samples, n_feats)
features = [features for features in dataframe["features"].values]
flatten = np.concatenate(features, axis=0)

lengths = np.array([...])

# train HMM
hmm.fit(X=flatten, lengths=lengths)
```

*For the HMM, it is necessary to choose a meaningful number of states. How many states (`n_components`) do you choose, and why?*

*With respect to the used `hmmlearn` library. How can you enforce a linear topology?*

*You might find that certain digits perform particularly bad; what could be a reason and how to mitigate it?*
    
2.3 Compute the [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix) for each speaker and for the overall dataset by combining the predictions of the cross-validation. You can use the [`scikit-learn`](https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html) library.

2.4 Additional experiment: Compare the results without and with per-speaker feature normalization. How does the performance change?

In [ ]:
### TODO:
### 1. set the `n_components` for all digits (choose a meaningful number of states)

n_comps = {i: None for i in DIGITS}

### YOUR CODE HERE



### END YOUR CODE

In [ ]:
### TODO: 
### 1. implement the 6-fold cross-validation loop
### 2. allocate and initialize the HMMs, one for each digit; set a linear topology
### 3. train the HMMs using the fit method; data needs to be concatenated
### 4. evaluate the trained models on the test speaker; how do you decide which word
###    was spoken?

### YOUR CODE HERE



### END YOUR CODE

In [ ]:
### TODO: 
### 1. based on the results, compute and display the confusion matrix for 
###    each test speaker 
### 2. compute and display the confusion matrix for the overall dataset

### YOUR CODE HERE



### END YOUR CODE

---

## Task 2) Decoding Sequences of Digits

The example above can't handle sequences of spoken digits.
In this part of the assignment, you'll build a basic decoder that is able to decode arbitrary sequences of digits (without a prior, though).
The `decode` method in `hmmlearn` only works for a single HMM.
There are two ways how to solve this assignment:

- Construct a "meta" HMM from the previously trained digit HMMs, by allowing state transitions from one digit to another; the resulting HMM can be decoded using the existing `decode` method (don't forget to re-map the state ids to the originating digit).

- (Optional) Implement a real (time-synchronous) decoder using beam search. The straight-forward way is to maintain a (sorted) list of active hypotheses (ie. state history and current log-likelihood) that is first expanded and then pruned in each time step. The tricky part is at the "end" of a model: do you loop or expand new words?

---

### Generate Test Sequences

3.1 Generate a few test sequences of random length in between 3 and 6 digits; use [`numpy.random.randint`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.randint.html) and be sure to also retain the digits sequence since we need to compute edit distance between reference and hypotheses later.

In [ ]:
def create_digit_sequence(speaker_dataframe, min_digits, max_digits):
    """
    Creates a sequence of spoken digits from a speaker and returns the
    features and reference label.
    """
    ### YOUR CODE HERE
    
    raise NotImplementedError()
    
    ### END YOUR CODE

In [ ]:
### Notice: just for test purposes

# speaker = "george"
# data_george = dataframe[dataframe["speaker"] == speaker]
# for i in range(20):
#     data_seq, digits = create_digit_sequence(data_george)
#     print("Digits: {}".format(digits))

### Create "meta" HMM

4.1 Combine the previously trained HMMs to a single "meta" HMM, altering the transition probabilities to make a circular graph that allows each word to follow another.

4.2 Implement a method that converts a state sequence relating to the meta HMM into a sequence of actual digits.

4.3 Decode your test sequences and compute the [word error rate](https://en.wikipedia.org/wiki/Word_error_rate) (WER) with [JiWER](https://pypi.org/project/jiwer/) (install the package in your working environment).

4.4 Compute an overall WER; ie. over the cross-validation.

4.5 (Optional) Implement a basic time-synchronous beam search; how do the results compare to the above viterbi decoding in terms of accuracy and time?

In [ ]:
### YOUR CODE HERE



### END YOUR CODE